# MinDivLP Summer Project
-- Chase Colbert, with much help and guidance from Dr. Simon Foucart, Dr. Ivan Ivanov, and Dr. David Koslicki

## Introduction

With the implementation of MinDivLP in MATLAB created for reconstruction of taxonomic profiles of microbiomes through the use of k-mer counts and compressive sensing methods, the purpose of this project is to port MinDivLP to Python and test it against other methods. The project began during the school year with receiving a research plan from Dr. Foucart, Dr. Ivanov, and Dr. Koslicki to implement the MinDivLP algorithm in Python, benchmark it against other methods of taxonomic profiling, and then apply it to real data.

## Preparation
During the rest of the semester, I spent time learning some basics of DNA sequencing and microbiome research; I read articles and went through tutorials recommended by Dr. Ivanov. The first article, [review of microbiome research done in the National Institiutes of Health](https://microbiomejournal.biomedcentral.com/articles/10.1186/s40168-019-0620-y), made clear the need and uses for taxonomic profiling, particularly for disease research. After reading this article, I watched the [first](https://www.youtube.com/watch?v=C_MaY8Lt4XM) and [second](https://www.youtube.com/watch?v=AvuTp3s_a1o) NCBI NOW (an online workshop for next generation sequencing) lectures. The first gave an overview of methods provided by NCBI, and the second gave an overview of the FASTQ file format, a format for sequence reads with quality scores, and how to use those quality scores to trim possibly inaccurate reads. The FASTA file is a similar format, but without quality scores, that is used most commonly throughout this project. After learning about these formats, I learned about QIIME2, a microbiome analysis package. Through pieces of the [QIIME2 Moving Pictures Tutorial](https://docs.qiime2.org/2020.6/tutorials/moving-pictures/) (after installing [QIIME2 with a VirtualBox](https://docs.qiime2.org/2020.6/install/virtual/virtualbox/)), I learned how to use QIIME2 to input FASTQ files, trim them, and assign taxonomy. 

## Implementation
To begin the work on porting MinDivLP to Python, I read through the [preprint](https://www.math.tamu.edu/~foucart/publi/Biodiversity.pdf) written by Dr. Foucart and Dr. Koslicki to further understand the $k$-mer sensing matrix $\mathbf{A}^{(k)}$, the population proportion vector $\mathbf{x}$, the sample $k$-mer proportion vector $\mathbf{y}^{(k)}$, and the problem, MinDivLP: 

$$\underset{x \in \mathbb{R} ^N}{\text{minimize }} \sum_{j=1}^{N} \frac{x_j}{\langle \mathbf{b}_j^{(h)} , \mathbf{y}^{(h)} \rangle ^{1-q}} \text{ subject to } \mathbf{A}^{(k)} \mathbf{x} = \mathbf{y}^{(k)} \text{ and } \mathbf{x} \geq 0,$$

where $q$ is a small constant and $\mathbf{b}_j^{(h)}$ is the $j$th column of the matrix $\mathbf{B}^{(h)}$ which equals $\mathbf{A^{(h)}}$ with every nonzero element changed to $1$. This problem can be replaced by a problem whose solutions converge to those of MinDivLP as $\lambda \to \infty$:

$$\underset{x \in \mathbb{R} ^N}{\text{minimize }} \left(\sum_{j=1}^{N} \frac{x_j}{\langle \mathbf{b}_j^{(h)} , \mathbf{y}^{(h)} \rangle ^{1-q}}\right)^2 + \lambda^2 \lVert \mathbf{A}^{(k)} \mathbf{x} - \mathbf{y}^{(k)} \rVert_2^2  \text{ subject to } \mathbf{x} \geq 0,$$

which, by a method similar to that done in Quikr, is equivalent to

$$\underset{x \in \mathbb{R} ^N}{\text{minimize }} \lVert \mathbf{\tilde{A}} \mathbf{x} - \mathbf{\tilde{y}} \rVert_2^2  \text{ subject to } \mathbf{x} \geq 0,$$

where

$$
\mathbf{\tilde{A}} = 
\left[
\begin{array}{c}
  \begin{array}{c c c}
     \frac{1}{\langle \mathbf{b}_1^{(h)} , \mathbf{y}^{(h)} \rangle ^{1-q}} 
     & \cdots 
     & \frac{1}{\langle \mathbf{b}_N^{(h)} , \mathbf{y}^{(h)} \rangle ^{1-q}} \\
  \end{array}\\
  \hline
  \lambda \mathbf{A}^{(k)}
 \end{array}
\right] \quad \text{and} \quad
\mathbf{\tilde{y}} = 
\left[
\begin{array}{c}
  0\\
  \hline
  \lambda \mathbf{y}^{(k)}
 \end{array}
\right].
$$
The importance of this equivalency is the solvability of this nonnegative least squares problem and the suitability of the [Lawson-Hanson nonnegative least squares algorithm](https://en.wikipedia.org/wiki/Non-negative_least_squares) for sparse recovery. Once I gained an understanding of these concepts, I created a [Python implementation](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/src/MinDivLP.py) of MinDivLP (through the proxy problem) with the help of the [MATLAB version](https://github.com/KoslickiLab/DiversityOptimization/blob/master/MatlabCode/src/MinDivLP.m). This implementation simply takes in the sensing matrices, sample $k$-mer proportion vectors, and constants, forms $\mathbf{\tilde{A}}$ and $\mathbf{\tilde{y}}$, and performs nonnegative least squares. Once I had implemented MinDivLP, I made a [Python version](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/scripts/MinDivLP_script.py) of Dr. Koslicki's [MATLAB test script](https://github.com/KoslickiLab/DiversityOptimization/blob/master/MatlabCode/src/MinDivLP_script.m) for MinDivLP and saw that both implementations gave the same answer in about the same amount of time. 
#### Note: 
Throughout the code, large_k is $h$, small_k is $k$, and, in the Python implementation, const is $\lambda$. Following this, A_k_large is $\mathbf{A}^{(h)}$, A_k_small is $\mathbf{A}^{(k)}$, y_large is $\mathbf{y}^{(h)}$, and y_small is $\mathbf{y}^{(k)}$. Also, $\mathbf{B}^{(h)}$ is simply B.

## Benchmarking
### Benchmarking Speed

After implementing the function, I created a [framework](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/experiments/Benchmark_speed.py) to benchmark the speed of the function with various values of $h$, support sizes of $\mathbf{x}$, and database size, i.e various values of $N$. The test was done on the Greengenes 97% OTUs database, which when full contains 99322 OTUs. Here is a single run of the test:

First we import some packages and MinDivLP and set the variables.

In [ ]:
import sys
import os
import time
import numpy as np
import scipy.io as sio
import numpy.random as rand
sys.path.append(os.path.abspath(".."))  # make sure python knows where to find MinDivLP
from src.MinDivLP import MinDivLP

num_cols = 1000
support_size = 25
large_k = 6
small_k = 4
const = 10000
q = 0.01
data_dir = "../data" # PythonCode/data

Sensing matrices are loaded through scipy.io from .mat files.

In [ ]:
## Import and prepare the data
A_k = sio.loadmat(f'{data_dir}/97_otus_subset.fasta_A_{large_k}.mat')
A_k_large = A_k['A_k']

A_k = sio.loadmat(f'{data_dir}/97_otus_subset.fasta_A_{small_k}.mat')
A_k_small = A_k['A_k']

del A_k

# Adjust to num_cols
A_k_large = A_k_large[:, 0:num_cols]
A_k_small = A_k_small[:, 0:num_cols]

In this benchmark, population proportions $\mathbf{x}$ are randomly generated from a support size, and the sample $k$-mer proportions $\mathbf{y}$ are truly equal to $\mathbf{Ax}$ with some random error.

In [ ]:
## Create the simulated ground truth
supp = rand.choice(range(0, support_size), size=(1, support_size), replace=False)  # location of the support
true_x = np.zeros((num_cols, 1))  # the true x vector we are trying to reconstruct
true_x[supp] = rand.random((support_size, 1))  # populate with random data
true_x = true_x / sum(true_x)  # normalize to be a probability vector

# Noiseless y-vectors
y_small_true = A_k_small @ true_x
y_large_true = A_k_large @ true_x

# Noisy y-vectors
noise_eps = .00001  # size of noise to add
y_small_noise = y_small_true + noise_eps * rand.random((A_k_small.shape[0], 1))
y_small_noise = y_small_noise / sum(y_small_noise)

Reconstruction is timed and then compared to the true value of $\mathbf{x}$.

In [ ]:
## Reconstruction
start_time = time.time()
x_star = MinDivLP(A_k_small, A_k_large, y_small_noise, y_large_true, 10000, 0.01).reshape(num_cols, 1)
print(f"Time: {time.time() - start_time} seconds")

print(f"L1 error: {sum(abs(x_star - true_x))}")

The process is essentially the same, except the artificial $\mathbf{x}$ and $\mathbf{y}$ vectors created, for later benchmarking tests.

#### Results
The time required increased primarily with the number of columns in the sensing matrix, but the times did not get out of hand. Also, the log of the time is fairly linear with respect to the log of the support size, as was expected. Here is one graph from the results:
<img src="https://user-images.githubusercontent.com/61798566/83340054-5c45dd80-a299-11ea-9f2a-857661e70ad7.png" width="700"/>

The rest of the results from this benchmark may be found in [issue #2](https://github.com/KoslickiLab/DiversityOptimization/issues/2).

#### Note:

Also, during this time, Dr. Koslicki created Python scripts to [create $k$-mer sensing matrices](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/src/Form16SSensingMatrix.py) and to [create $\mathbf{y}$ vectors](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/src/Form16SyVector.py) in [issue #3](https://github.com/KoslickiLab/DiversityOptimization/issues/3) and [issue #4](https://github.com/KoslickiLab/DiversityOptimization/issues/4), respectively. This was used to create sensing matrices in this benchmark and is ubiquitous in the rest of the project.

### Simulated Benchmark

Now that the function was verified to work on noisy $\mathbf{y}$ vectors that are definitely approximately equal to $\mathbf{Ax}$, we went on to test the function using simulated reads, where $\mathbf{y}$ vectors are not derived from $\mathbf{A}$ and $\mathbf{x}$, but from simulated sampling of a group of metagenomes. Using a Java package that can be used to simulate sequencing reads called [BBMap](https://sourceforge.net/projects/bbmap/) and Dr. Koslicki's recommended BBMap function calls, I created a script to evaluate MinDivLP using the $\mathbf{y}$ vectors created from the output of BBMap, a FASTA file, with varying values for $k$, $h$, $q$, $\lambda$, support size, and coverage of the simulated sequencing. After doing so, Dr. Koslicki modified it to make [faster version](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/experiments/simulated_benchmark/simulated_benchmark_fast.py), ordering the loops more optimally.

#### Results

After running these tests, we found that, because of BBMap's sampling, $\mathbf{Ax}$ was not matching $\mathbf{y}$. Dr. Koslicki determined that the error came from bbmap sampling more toward the center of database genomes. Thus we decided that the L1 error of the results may not be indicative of performance with real reads.
Also, Dr. Koslicki found in his runs that the performance of MinDivLP is sensitive to lambda, and that MinDivLP indeed performed better than Quikr, an important result.

<table> <tr>
<td> <img src="https://user-images.githubusercontent.com/6362936/84073250-49ac6200-a985-11ea-990b-c417c0b9f90c.png" width="400"/> </td>
<td> <img src="https://user-images.githubusercontent.com/6362936/84079370-5cc42f80-a98f-11ea-9374-c5a1ff548530.png" width="400"/> </td>
</tr> </table>
    
Also, it should be noted that even with the error introduced, with increases in $k$, i.e. small_k,  L1 error decreased significantly and the proportion of the true support size included in the reconstructed support size increased. 
<img src="https://user-images.githubusercontent.com/61798566/84158734-d3196e00-aa31-11ea-9a0c-af7a176b5f2b.png" style="width:700px"/>


More graphs and details can be found in [issue #12](https://github.com/KoslickiLab/DiversityOptimization/issues/12).

### Benchmark vs Others (in progress)

At this stage, we sought to get an idea of how MinDivLP compares to other methods of taxonomic profiling from reads and a sequence database, particularly Blast+ and Naive-Bayes. These methods are included in the QIIME2 package; this is why I was familiarized with QIIME2 in the Moving Pictures Tutorial.

#### Preparation for comparison

To compare MinDivLP to other methods, there needs to be a way to convert the reconstructed vector $\mathbf{x}$ to a taxonomic profile using the reference database. To do this, I wrote a [script](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/src/ConvertXToTaxonomicProfile.py) that takes the reconstructed vector and the reference database's taxonomic profiles to create a TSV file that lists the corresponding ID, value, and taxonomy of each nonzero element of the vector. Then, with a command line input, the TSV file can be converted to the standard [BIOM format](https://biom-format.org/) for comparison with other methods. More details on the creation of this script can be found in [issue #7](https://github.com/KoslickiLab/DiversityOptimization/issues/7).

#### Results

##### Mock Community
For our purposes, taxon accuracy rate and taxon detection rate are the applicable measures. Taxon accuracy rate at a particular taxonomic level is the proportion of observed (reconstructed) taxa that were expected. Taxon detection rate at a particular taxonomic level is the proportion of expected taxa that are observed.

Here are the results of MinDivLP, in bright green, against other classifiers on several mock communities $k = 8$ and $k = 10$, $h = 12$, $q = 0.01$, and $\lambda = 10000$:

<table> <tr>
<td> <img src="https://user-images.githubusercontent.com/61798566/87084365-bb313780-c1f3-11ea-966b-4b66615e65c5.png" width="200"/> </td>
<td> <img src="https://user-images.githubusercontent.com/61798566/87084374-bd939180-c1f3-11ea-9441-d2154feaeeb7.png" width="200"/> </td>
</tr> </table>

It is immediately noticeable that the taxon accuracy rate is much lower than other classifiers. This is because of many false positives from not having a threshold for the values of $\mathbf{x}$. To improve accuracy, a threshold of $0.001$ was added by setting elements of the reconstructed $\mathbf{x}$ to zero if they are less than $0.001$. This change gave the following result with thresholded MinDivLP in burnt orange and non-thresholded MinDivLP in bright green:

<table> <tr>
<td> <img src="https://user-images.githubusercontent.com/61798566/87264366-6c023580-c485-11ea-95d7-4a25ad3fffc4.png" width="200"/> </td>
<td> <img src="https://user-images.githubusercontent.com/61798566/87264375-6f95bc80-c485-11ea-91b5-9ba534685e7c.png" width="200"/> </td>
</tr> </table>

There was much improvement in accuracy at the expense of some detection. After testing with increased $k$ values, further testing may be necessary to see if the threshold of $0.001$ could be improved upon.

##### Runtime Analysis
Here are the results of the runtime analysis provided in tax-credit's framework:

<table> <tr>
<td> <img src="https://user-images.githubusercontent.com/61798566/87572174-c6bbad00-c690-11ea-904e-bca21eb438af.png" width="400"/> </td>
<td> <img src="https://user-images.githubusercontent.com/61798566/87572183-c8857080-c690-11ea-85de-b4a11566a047.png" width="400"/> </td>
</tr> </table>

This analysis does not show the strength of MinDivLP which assumes a relatively small query database, at least in terms of diversity. I am interested in running a runtime analysis with around 50 or 75 sequences in the query database.

More details can be found in [issue #9](https://github.com/KoslickiLab/DiversityOptimization/issues/9).

#### A note on nonnegative least squares with sparse matrices:

Up until Benchmark vs Others, the Python implementation of MinDivLP used [scipy.optimize.nnls](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.nnls.html), a Python wrapper of the Lawson-Hanson algorithm written in Fortran, which does not allow for the input of sparse matrices. Large $k$ values give better reconstruction accuracy at the expense of increasing the size of the sensing matrix. If stored in a dense format, the sensing matrix $\mathbf{A}^{(k)}$ and thus the input $\mathbf{\tilde{A}}$, will be infeasibly large for large enough $k$ values (values that are low when considering the comparitively poor reconstruction accuracy they produce). When I realized this problem was present, I spent a good deal of time working on a solution, a way to compute nonnegative least squares with a scipy sparse matrix. I originally tried methods such as [scipy.optimize.lsq_linear](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.lsq_linear.html) to no avail. Then I wrote a Python implementation of the Lawson-Hanson algorithm, [sparse_nnls](https://github.com/KoslickiLab/DiversityOptimization/blob/master/PythonCode/src/sparse_nnls.py),  based on Dr. Koslicki's [Julia implementation](https://github.com/dkoslicki/Quikr/blob/master/src/Julia/lsqnonneg.jl), but a mistake in indentation with no error and poor testing on my part led me to believe the inner function I used for solving an overdetermined sparse linear system, [scipy.sparse.linalg.lsqr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lsqr.html#scipy.sparse.linalg.lsqr), was too slow. So I spent several days working on finding or creating a solution through sparse C packages and Cython, particularly [SuiteSparse](http://faculty.cse.tamu.edu/davis/suitesparse.html). Once I realized my mistake, however, the Python implementation worked quite well, giving nearly the same answers (L1 difference of less than $10^{-5}$ on the comparisons done) as scipy.optimize.nnls. 

These problems were addressed in [issue #14](https://github.com/KoslickiLab/DiversityOptimization/issues/14).

## Future Work

It may be helpful to convert the vectors $\mathbf{x}$ and $\mathbf{y}$ to sparse matrices to save memory.

Looking into other options for nonnegative least squares algorithms could be beneficial since others tout better speeds.

Trying larger $h$ values hopefully will yield accuracy rates more on par with those of other methods.

Testing on real data still has yet to be done.